# Job-candidate matching

In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
import os

C:\Users\IuG_Lap1\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


##### 1. Load datasets

In [2]:
cv_df = pd.read_csv("cv_records.csv")
job_df = pd.read_csv("job_records.csv")

cv_id_to_idx = dict(zip(cv_df["cv_id"], range(len(cv_df))))
job_id_to_idx = dict(zip(job_df["job_id"], range(len(job_df))))

In [3]:
# Build all CV-Job posting matched pairs (same ROLE)
from collect_cv_job_pairs import collect_cv_job_pairs

job_pairs = collect_cv_job_pairs(cv_df, job_df)

Total matched pairs: 8640


In [4]:
out_dir = "similarity_matching_output"

## JobBERT-v2

https://huggingface.co/TechWolf/JobBERT-v2

##### Load model and encode texts

In [5]:
model = SentenceTransformer("TechWolf/JobBERT-v2")

In [6]:
cv_embeddings_v2 = model.encode(cv_df["description"].tolist(), normalize_embeddings=True)
job_embeddings_v2 = model.encode(job_df["text"].tolist(), normalize_embeddings=True)

##### Compute similarity only for matched pairs

In [7]:
records = []

for cv_id, job_id in job_pairs:
    cv_idx = cv_id_to_idx[cv_id]
    job_idx = job_id_to_idx[job_id]

    sim = util.cos_sim(
        cv_embeddings_v2[cv_idx],
        job_embeddings_v2[job_idx]
    ).item()

    records.append({
        "cv_id": cv_id,
        "name": cv_df.loc[cv_idx, "name"],
        "gender": cv_df.loc[cv_idx, "gender"],
        "ethnicity": cv_df.loc[cv_idx, "ethnicity"],
        "role": cv_df.loc[cv_idx, "role"],

        "job_id": job_id,
        "job_role": job_df.loc[job_idx, "role"],
        "job_domain": job_df.loc[job_idx, "domain"] if "domain" in job_df.columns else None,
        "job_level": job_df.loc[job_idx, "level"],
        "job_text": job_df.loc[job_idx, "text"],

        "similarity": sim
    })

out_df = pd.DataFrame(records)
out_path = os.path.join(out_dir, "cv_job_similarity_jb_v2.csv")
out_df.to_csv(out_path, index=False)

print(f"Saved → {out_path}")
out_df.head()

Saved → similarity_matching_output\cv_job_similarity_jb_v2.csv


,cv_id,name,gender,ethnicity,role,job_id,job_role,job_domain,job_level,job_text,similarity
0,CV_0,Ahmed Hassan,male,arabic_middle_eastern,Software Engineer,JOB_0,Software Engineer,tech,Junior,Hiring a Junior Software Engineer to implement...,0.560831
1,CV_0,Ahmed Hassan,male,arabic_middle_eastern,Software Engineer,JOB_1,Software Engineer,tech,Junior,Junior Software Engineer role focused on contr...,0.603085
2,CV_0,Ahmed Hassan,male,arabic_middle_eastern,Software Engineer,JOB_2,Software Engineer,tech,Mid,"Seeking a Software Engineer to design APIs, ma...",0.695089
3,CV_0,Ahmed Hassan,male,arabic_middle_eastern,Software Engineer,JOB_3,Software Engineer,tech,Mid,Software Engineer needed to work on applicatio...,0.582181
4,CV_0,Ahmed Hassan,male,arabic_middle_eastern,Software Engineer,JOB_4,Software Engineer,tech,Senior,Senior Software Engineer role involving owners...,0.568125


---

## JobBERT-v3

https://huggingface.co/TechWolf/JobBERT-v3

##### Load model and encode texts

In [8]:
model = SentenceTransformer("TechWolf/JobBERT-v3")

In [9]:
cv_embeddings_v3 = model.encode(cv_df["description"].tolist(), normalize_embeddings=True)
job_embeddings_v3 = model.encode(job_df["text"].tolist(), normalize_embeddings=True)

##### Compute similarity only for matched pairs

In [10]:
records = []

for cv_id, job_id in job_pairs:
    cv_idx = cv_id_to_idx[cv_id]
    job_idx = job_id_to_idx[job_id]

    sim = util.cos_sim(
        cv_embeddings_v3[cv_idx],
        job_embeddings_v3[job_idx]
    ).item()

    records.append({
        "cv_id": cv_id,
        "name": cv_df.loc[cv_idx, "name"],
        "gender": cv_df.loc[cv_idx, "gender"],
        "ethnicity": cv_df.loc[cv_idx, "ethnicity"],
        "role": cv_df.loc[cv_idx, "role"],

        "job_id": job_id,
        "job_role": job_df.loc[job_idx, "role"],
        "job_domain": job_df.loc[job_idx, "domain"] if "domain" in job_df.columns else None,
        "job_level": job_df.loc[job_idx, "level"],
        "job_text": job_df.loc[job_idx, "text"],

        "similarity": sim
    })

out_df = pd.DataFrame(records)
out_path = os.path.join(out_dir, "cv_job_similarity_jb_v3.csv")
out_df.to_csv(out_path, index=False)

print(f"Saved → {out_path}")
out_df.head()

Saved → similarity_matching_output\cv_job_similarity_jb_v3.csv


,cv_id,name,gender,ethnicity,role,job_id,job_role,job_domain,job_level,job_text,similarity
0,CV_0,Ahmed Hassan,male,arabic_middle_eastern,Software Engineer,JOB_0,Software Engineer,tech,Junior,Hiring a Junior Software Engineer to implement...,0.460334
1,CV_0,Ahmed Hassan,male,arabic_middle_eastern,Software Engineer,JOB_1,Software Engineer,tech,Junior,Junior Software Engineer role focused on contr...,0.379399
2,CV_0,Ahmed Hassan,male,arabic_middle_eastern,Software Engineer,JOB_2,Software Engineer,tech,Mid,"Seeking a Software Engineer to design APIs, ma...",0.484444
3,CV_0,Ahmed Hassan,male,arabic_middle_eastern,Software Engineer,JOB_3,Software Engineer,tech,Mid,Software Engineer needed to work on applicatio...,0.549095
4,CV_0,Ahmed Hassan,male,arabic_middle_eastern,Software Engineer,JOB_4,Software Engineer,tech,Senior,Senior Software Engineer role involving owners...,0.490050


---

## Paraphrase-multilingual-MiniLM-L12 V2 

A job matching sentence-transformers model finetuned from **sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2**
https://huggingface.co/forestav/job_matching_sentence_transformer

##### Load model and encode texts

In [11]:
model_2 = SentenceTransformer("forestav/job_matching_sentence_transformer")

In [12]:
cv_embeddings_minilm = model_2.encode(cv_df["description"].tolist(), normalize_embeddings=True)
job_embeddings_minilm = model_2.encode(job_df["text"].tolist(), normalize_embeddings=True)

In [13]:
records = []

for cv_id, job_id in job_pairs:
    cv_idx = cv_id_to_idx[cv_id]
    job_idx = job_id_to_idx[job_id]

    sim = util.cos_sim(
        cv_embeddings_minilm[cv_idx],
        job_embeddings_minilm[job_idx]
    ).item()

    records.append({
        "cv_id": cv_id,
        "name": cv_df.loc[cv_idx, "name"],
        "gender": cv_df.loc[cv_idx, "gender"],
        "ethnicity": cv_df.loc[cv_idx, "ethnicity"],
        "role": cv_df.loc[cv_idx, "role"],

        "job_id": job_id,
        "job_role": job_df.loc[job_idx, "role"],
        "job_domain": job_df.loc[job_idx, "domain"] if "domain" in job_df.columns else None,
        "job_level": job_df.loc[job_idx, "level"],
        "job_text": job_df.loc[job_idx, "text"],

        "similarity": sim
    })

out_df = pd.DataFrame(records)
out_path = os.path.join(out_dir, "cv_job_similarity_minilm_base.csv")
out_df.to_csv(out_path, index=False)

print(f"Saved → {out_path}")
out_df.head()

Saved → similarity_matching_output\cv_job_similarity_minilm_base.csv


,cv_id,name,gender,ethnicity,role,job_id,job_role,job_domain,job_level,job_text,similarity
0,CV_0,Ahmed Hassan,male,arabic_middle_eastern,Software Engineer,JOB_0,Software Engineer,tech,Junior,Hiring a Junior Software Engineer to implement...,0.305610
1,CV_0,Ahmed Hassan,male,arabic_middle_eastern,Software Engineer,JOB_1,Software Engineer,tech,Junior,Junior Software Engineer role focused on contr...,0.379118
2,CV_0,Ahmed Hassan,male,arabic_middle_eastern,Software Engineer,JOB_2,Software Engineer,tech,Mid,"Seeking a Software Engineer to design APIs, ma...",0.528967
3,CV_0,Ahmed Hassan,male,arabic_middle_eastern,Software Engineer,JOB_3,Software Engineer,tech,Mid,Software Engineer needed to work on applicatio...,0.442899
4,CV_0,Ahmed Hassan,male,arabic_middle_eastern,Software Engineer,JOB_4,Software Engineer,tech,Senior,Senior Software Engineer role involving owners...,0.340402


---

## Fairness LoRA 

Fairness-aware LoRA adapter for resume–job matching built on top of **BAAI/bge-large-en-v1.5** \
https://huggingface.co/renhehuang/fair-resume-job-matcher-lora

In [14]:
from transformers import AutoTokenizer, AutoModel
from peft import PeftModel
import torch
import torch.nn.functional as F

BASE = "BAAI/bge-large-en-v1.5"
ADAPTER = "renhehuang/fair-resume-job-matcher-lora"  # HF repo id

tokenizer = AutoTokenizer.from_pretrained(BASE)
base_model = AutoModel.from_pretrained(BASE)

# Load LoRA adapter from Hugging Face
model_bge = PeftModel.from_pretrained(base_model, ADAPTER)
model_bge.eval();

In [15]:
def encode_batch(texts, max_length: int = 256):
    """Encode a batch of texts into L2-normalized embeddings (torch.Tensor)."""
    enc = tokenizer(
        texts,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=max_length,
    )
    enc = {k: v.to("cpu") for k, v in enc.items()}

    with torch.no_grad():
        out = model_bge(**enc)
        hidden = out.last_hidden_state                      # (batch, seq_len, hidden)
        emb = hidden.mean(dim=1)                            # simple mean pooling
        emb = F.normalize(emb, p=2, dim=1)                  # L2 norm

    return emb  # (batch, hidden)

def encode_all(texts, batch_size: int = 16, max_length: int = 256):
    """Encode an iterable of texts to a single (N, d) tensor."""
    all_embs = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i : i + batch_size]
        batch_embs = encode_batch(batch_texts, max_length=max_length)
        all_embs.append(batch_embs.cpu())
    return torch.cat(all_embs, dim=0)   # (N, d)

In [16]:
cv_embeddings_bge = encode_all(cv_df["description"].tolist(), batch_size=16)
job_embeddings_bge = encode_all(job_df["text"].tolist(), batch_size=16)

In [17]:
records = []

for cv_id, job_id in job_pairs:
    cv_idx = cv_id_to_idx[cv_id]
    job_idx = job_id_to_idx[job_id]

    cv_emb = cv_embeddings_bge[cv_idx]
    job_emb = job_embeddings_bge[job_idx]

    # embeddings are already normalized → dot product = cosine similarity
    sim = torch.dot(cv_emb, job_emb).item()

    # if you want the "probability" style score they use in the example:
    # prob = torch.sigmoid(torch.tensor(sim)).item()

    records.append({
        "cv_id": cv_id,
        "name": cv_df.loc[cv_idx, "name"],
        "gender": cv_df.loc[cv_idx, "gender"],
        "ethnicity": cv_df.loc[cv_idx, "ethnicity"],
        "role": cv_df.loc[cv_idx, "role"],

        "job_id": job_id,
        "job_role": job_df.loc[job_idx, "role"],
        "job_domain": job_df.loc[job_idx, "domain"] if "domain" in job_df.columns else None,
        "job_level": job_df.loc[job_idx, "level"],
        "job_text": job_df.loc[job_idx, "text"],

        "similarity": sim,
        # "prob": prob,              # uncomment if you want this column too
    })

out_df = pd.DataFrame(records)
out_path = os.path.join(out_dir, "cv_job_similarity_fair_lora.csv")
out_df.to_csv(out_path, index=False)

print(f"Saved → {out_path}")
out_df.head()

Saved → similarity_matching_output\cv_job_similarity_fair_lora.csv


,cv_id,name,gender,ethnicity,role,job_id,job_role,job_domain,job_level,job_text,similarity
0,CV_0,Ahmed Hassan,male,arabic_middle_eastern,Software Engineer,JOB_0,Software Engineer,tech,Junior,Hiring a Junior Software Engineer to implement...,0.654333
1,CV_0,Ahmed Hassan,male,arabic_middle_eastern,Software Engineer,JOB_1,Software Engineer,tech,Junior,Junior Software Engineer role focused on contr...,0.677761
2,CV_0,Ahmed Hassan,male,arabic_middle_eastern,Software Engineer,JOB_2,Software Engineer,tech,Mid,"Seeking a Software Engineer to design APIs, ma...",0.766032
3,CV_0,Ahmed Hassan,male,arabic_middle_eastern,Software Engineer,JOB_3,Software Engineer,tech,Mid,Software Engineer needed to work on applicatio...,0.709501
4,CV_0,Ahmed Hassan,male,arabic_middle_eastern,Software Engineer,JOB_4,Software Engineer,tech,Senior,Senior Software Engineer role involving owners...,0.687694


---

## Open AI text-embedding-3-small
https://platform.openai.com/docs/guides/embeddings

In [18]:
from openai import OpenAI
from dotenv import load_dotenv
import torch

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

In [19]:
def get_embedding(text, model="text-embedding-3-small"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input = [text], model=model).data[0].embedding

In [20]:
cv_embeddings_openai = cv_df["description"].apply(lambda x: get_embedding(x, model='text-embedding-3-small'))
job_embeddings_openai = job_df["text"].apply(lambda x: get_embedding(x, model='text-embedding-3-small'))

In [21]:
cv_embeddings_openai = [
    torch.tensor(e, dtype=torch.float32) for e in cv_embeddings_openai
]

job_embeddings_openai = [
    torch.tensor(e, dtype=torch.float32) for e in job_embeddings_openai
]

In [22]:
records = []

for cv_id, job_id in job_pairs:
    cv_idx = cv_id_to_idx[cv_id]
    job_idx = job_id_to_idx[job_id]

    cv_emb = cv_embeddings_openai[cv_idx]
    job_emb = job_embeddings_openai[job_idx]

    # embeddings are already normalized → dot product = cosine similarity
    sim = torch.dot(cv_emb, job_emb).item()

    # if you want the "probability" style score they use in the example:
    # prob = torch.sigmoid(torch.tensor(sim)).item()

    records.append({
        "cv_id": cv_id,
        "name": cv_df.loc[cv_idx, "name"],
        "gender": cv_df.loc[cv_idx, "gender"],
        "ethnicity": cv_df.loc[cv_idx, "ethnicity"],
        "role": cv_df.loc[cv_idx, "role"],

        "job_id": job_id,
        "job_role": job_df.loc[job_idx, "role"],
        "job_domain": job_df.loc[job_idx, "domain"] if "domain" in job_df.columns else None,
        "job_level": job_df.loc[job_idx, "level"],
        "job_text": job_df.loc[job_idx, "text"],

        "similarity": sim,
        # "prob": prob,              # uncomment if you want this column too
    })

out_df = pd.DataFrame(records)
out_path = os.path.join(out_dir, "cv_job_similarity_openai.csv")
out_df.to_csv(out_path, index=False)

print(f"Saved → {out_path}")
out_df.head()

Saved → similarity_matching_output\cv_job_similarity_openai.csv


,cv_id,name,gender,ethnicity,role,job_id,job_role,job_domain,job_level,job_text,similarity
0,CV_0,Ahmed Hassan,male,arabic_middle_eastern,Software Engineer,JOB_0,Software Engineer,tech,Junior,Hiring a Junior Software Engineer to implement...,0.335004
1,CV_0,Ahmed Hassan,male,arabic_middle_eastern,Software Engineer,JOB_1,Software Engineer,tech,Junior,Junior Software Engineer role focused on contr...,0.502206
2,CV_0,Ahmed Hassan,male,arabic_middle_eastern,Software Engineer,JOB_2,Software Engineer,tech,Mid,"Seeking a Software Engineer to design APIs, ma...",0.483109
3,CV_0,Ahmed Hassan,male,arabic_middle_eastern,Software Engineer,JOB_3,Software Engineer,tech,Mid,Software Engineer needed to work on applicatio...,0.424171
4,CV_0,Ahmed Hassan,male,arabic_middle_eastern,Software Engineer,JOB_4,Software Engineer,tech,Senior,Senior Software Engineer role involving owners...,0.439466
